In [1]:
import tensorflow as tf
import os, warnings
import pandas as pd
warnings.filterwarnings('ignore')
import numpy as np
from logp.zq.io.smi2feat import load_feat_from_csv
from logp.zq.io.load_dataframe import load_pd
from logp.zq.model.model import potential_model
from sklearn.metrics import mean_absolute_error,mean_squared_error
from logp.zq.io.base import sparse_batch

In [2]:
filename = 'logp/DATASETS/sf_ave_PhysProp_MD_weights_charge.csv'
dataset = pd.read_csv(filename)
dataset.shape

(4802, 4)

In [3]:
results = load_pd(filename)

In [4]:
# load the predicting data in martel
filename_martel = 'logp/DATASETS/Martel/MD_ave/weight_q/sf_ave_Martel_MD_weights_charge.csv'
dataset_martel = pd.read_csv(filename_martel)
dataset_martel.shape
results_martel = load_pd(filename_martel)
dataset_martel = lambda: load_feat_from_csv(results_martel,24,split={'train':10,'test':0},shuffle=False)
martel_train = lambda: dataset_martel()['train'].apply(sparse_batch(5))

In [5]:
# load the predicting data in star
filename_star = 'logp/DATASETS/star_nonstar/MD_ave/weight_q/sf_ave_star_MD_weights_charge.csv'
dataset_star = pd.read_csv(filename_star)
dataset_star.shape
results_star = load_pd(filename_star)

dataset_star = lambda: load_feat_from_csv(results_star,24,split={'train':10,'test':0},shuffle=False)
star_train = lambda: dataset_star()['train'].apply(sparse_batch(5))

In [6]:
# load the predicting data in huusk
filename_nonstar = 'logp/DATASETS/Huuskonen/MD_ave/weight_q/sf_ave_Huuskonen_MD_weights_charge.csv'
dataset_nonstar = pd.read_csv(filename_nonstar)
dataset_nonstar.shape
results_nonstar = load_pd(filename_nonstar)

dataset_nonstar = lambda: load_feat_from_csv(results_nonstar,24,split={'train':10,'test':0},shuffle=False)
nonstar_train = lambda: dataset_nonstar()['train'].apply(sparse_batch(5))

In [7]:
logps_martel = []
logps_star = []
logps_nonstar = []

for x in range(10):
    # define the parameters
    params = {
    'model_dir':'MD_ave_weight_q_PhysProp/100_100_100/lr_1e-3/kfold_{}'.format(x),
    'network':'bpnn',
    'network_params':{
    'nn_spec':{1:[100,100,100],6:[100,100,100],7:[100,100,100],8:[100,100,100]}},
    'model_params': {'learning_rate': 1e-3}
    }
    #split the dataset into train and test
    ftrain = 'logp/DATASETS/PhysProps/MD_ave/weight_q/sf_ave_PhysProp_MD_weights_q_train_{}.csv'.format(x)
    ftest  = 'logp/DATASETS/PhysProps/MD_ave/weight_q/sf_ave_PhysProp_MD_weights_q_test_{}.csv'.format(x)
    dtrain = load_pd(ftrain)
    dtest = load_pd(ftest)
    dataset_train = lambda: load_feat_from_csv(dtrain,24,split={'train':10,'test':0})
    dataset_test = lambda: load_feat_from_csv(dtest,24,split={'train':0,'test':10})
    # yeild the train and test `input_fn`
    train = lambda: dataset_train()['train'].repeat().shuffle(1000).apply(sparse_batch(100))
    test = lambda: dataset_test()['test'].repeat().apply(sparse_batch(50))
    
    # initialize the model
    model = potential_model(params)
    # define the hooks
    early_stopping = tf.estimator.experimental.stop_if_no_decrease_hook(model, metric_name='loss',
                                                              max_steps_without_decrease=1000,
                                                              min_steps=100)
    # train and evaluate the model
    train_spec = tf.estimator.TrainSpec(input_fn=train, hooks=[early_stopping] ,max_steps=200000)
    eval_spec = tf.estimator.EvalSpec(input_fn=test, steps=400)
    tf.estimator.train_and_evaluate(model, train_spec, eval_spec)
    
    # predict the data in martel 
    preds_martel = model.predict(input_fn=martel_train)
    preds_star = model.predict(input_fn=star_train)
    preds_nonstar = model.predict(input_fn=nonstar_train)
    
    logp_martel = []
    for pred in preds_martel:
        logp_martel.append(pred['logp'])
    logps_martel.append(logp_martel)
    
    logp_star = []
    for pred in preds_star:
        logp_star.append(pred['logp'])
    logps_star.append(logp_star)
    
    logp_nonstar = []
    for pred in preds_nonstar:
        logp_nonstar.append(pred['logp'])
    logps_nonstar.append(logp_nonstar)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'MD_ave_weight_q_PhysProp/100_100_100/lr_1e-3/kfold_0', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Not using Distribute Coordina

INFO:tensorflow:loss = 0.33266807, step = 4900 (0.973 sec)
INFO:tensorflow:global_step/sec: 102.843
INFO:tensorflow:loss = 0.51498556, step = 5000 (0.973 sec)
INFO:tensorflow:global_step/sec: 103.427
INFO:tensorflow:loss = 0.35694167, step = 5100 (0.967 sec)
INFO:tensorflow:global_step/sec: 103.015
INFO:tensorflow:loss = 0.3998088, step = 5200 (0.971 sec)
INFO:tensorflow:global_step/sec: 102.215
INFO:tensorflow:loss = 0.23134592, step = 5300 (0.978 sec)
INFO:tensorflow:global_step/sec: 102.759
INFO:tensorflow:loss = 0.3119379, step = 5400 (0.973 sec)
INFO:tensorflow:global_step/sec: 103.043
INFO:tensorflow:loss = 0.25585285, step = 5500 (0.970 sec)
INFO:tensorflow:global_step/sec: 102.997
INFO:tensorflow:loss = 0.24897508, step = 5600 (0.971 sec)
INFO:tensorflow:global_step/sec: 103.286
INFO:tensorflow:loss = 0.31849203, step = 5700 (0.968 sec)
INFO:tensorflow:global_step/sec: 101.792
INFO:tensorflow:loss = 0.215771, step = 5800 (0.982 sec)
INFO:tensorflow:global_step/sec: 104.27
INFO:

INFO:tensorflow:loss = 0.15581965, step = 13100 (0.973 sec)
INFO:tensorflow:global_step/sec: 104.583
INFO:tensorflow:loss = 0.11591415, step = 13200 (0.956 sec)
INFO:tensorflow:global_step/sec: 102.985
INFO:tensorflow:loss = 0.23997131, step = 13300 (0.971 sec)
INFO:tensorflow:global_step/sec: 103.92
INFO:tensorflow:loss = 0.2188201, step = 13400 (0.962 sec)
INFO:tensorflow:global_step/sec: 104.523
INFO:tensorflow:loss = 0.14502034, step = 13500 (0.957 sec)
INFO:tensorflow:global_step/sec: 103.592
INFO:tensorflow:loss = 0.19877958, step = 13600 (0.965 sec)
INFO:tensorflow:global_step/sec: 103.593
INFO:tensorflow:loss = 0.13997383, step = 13700 (0.965 sec)
INFO:tensorflow:global_step/sec: 103.252
INFO:tensorflow:loss = 0.15220414, step = 13800 (0.968 sec)
INFO:tensorflow:global_step/sec: 102.245
INFO:tensorflow:loss = 0.11702338, step = 13900 (0.978 sec)
INFO:tensorflow:global_step/sec: 103.166
INFO:tensorflow:loss = 0.19960396, step = 14000 (0.969 sec)
INFO:tensorflow:global_step/sec: 

INFO:tensorflow:global_step/sec: 102.762
INFO:tensorflow:loss = 0.06611186, step = 21300 (0.974 sec)
INFO:tensorflow:global_step/sec: 103.168
INFO:tensorflow:loss = 0.0944139, step = 21400 (0.969 sec)
INFO:tensorflow:global_step/sec: 103.201
INFO:tensorflow:loss = 0.07359415, step = 21500 (0.969 sec)
INFO:tensorflow:global_step/sec: 102.931
INFO:tensorflow:loss = 0.098707885, step = 21600 (0.971 sec)
INFO:tensorflow:global_step/sec: 101.78
INFO:tensorflow:loss = 0.091742955, step = 21700 (0.982 sec)
INFO:tensorflow:global_step/sec: 102.921
INFO:tensorflow:loss = 0.118737295, step = 21800 (0.972 sec)
INFO:tensorflow:global_step/sec: 103.535
INFO:tensorflow:loss = 0.07812633, step = 21900 (0.966 sec)
INFO:tensorflow:global_step/sec: 102.47
INFO:tensorflow:loss = 0.11459008, step = 22000 (0.976 sec)
INFO:tensorflow:global_step/sec: 103.416
INFO:tensorflow:loss = 0.14220297, step = 22100 (0.967 sec)
INFO:tensorflow:global_step/sec: 102.766
INFO:tensorflow:loss = 0.115689136, step = 22200 (

INFO:tensorflow:global_step/sec: 103.396
INFO:tensorflow:loss = 0.05770058, step = 29400 (0.967 sec)
INFO:tensorflow:global_step/sec: 103.075
INFO:tensorflow:loss = 0.0377502, step = 29500 (0.970 sec)
INFO:tensorflow:global_step/sec: 103.622
INFO:tensorflow:loss = 0.03417707, step = 29600 (0.965 sec)
INFO:tensorflow:global_step/sec: 102.742
INFO:tensorflow:loss = 0.042706918, step = 29700 (0.973 sec)
INFO:tensorflow:global_step/sec: 103.559
INFO:tensorflow:loss = 0.044647455, step = 29800 (0.966 sec)
INFO:tensorflow:global_step/sec: 103.223
INFO:tensorflow:loss = 0.048679482, step = 29900 (0.969 sec)
INFO:tensorflow:global_step/sec: 103
INFO:tensorflow:loss = 0.038710084, step = 30000 (0.970 sec)
INFO:tensorflow:global_step/sec: 103.689
INFO:tensorflow:loss = 0.03300099, step = 30100 (0.964 sec)
INFO:tensorflow:global_step/sec: 103.271
INFO:tensorflow:loss = 0.042175125, step = 30200 (0.968 sec)
INFO:tensorflow:global_step/sec: 103.862
INFO:tensorflow:loss = 0.047540847, step = 30300 (

...

INFO:tensorflow:global_step/sec: 106.776
INFO:tensorflow:loss = 0.022658959, step = 42100 (0.937 sec)
INFO:tensorflow:global_step/sec: 105.012
INFO:tensorflow:loss = 0.018233875, step = 42200 (0.952 sec)
INFO:tensorflow:global_step/sec: 106.496
INFO:tensorflow:loss = 0.021480095, step = 42300 (0.939 sec)
INFO:tensorflow:global_step/sec: 105.08
INFO:tensorflow:loss = 0.019661497, step = 42400 (0.952 sec)
INFO:tensorflow:global_step/sec: 105.38
INFO:tensorflow:loss = 0.039090067, step = 42500 (0.949 sec)
INFO:tensorflow:global_step/sec: 106.508
INFO:tensorflow:loss = 0.021893132, step = 42600 (0.939 sec)
INFO:tensorflow:global_step/sec: 106.108
INFO:tensorflow:loss = 0.022471404, step = 42700 (0.942 sec)
INFO:tensorflow:global_step/sec: 105.39
INFO:tensorflow:loss = 0.024900472, step = 42800 (0.949 sec)
INFO:tensorflow:global_step/sec: 104.997
INFO:tensorflow:loss = 0.019668639, step = 42900 (0.952 sec)
INFO:tensorflow:global_step/sec: 106.318
INFO:tensorflow:loss = 0.02402506, step = 43

INFO:tensorflow:global_step/sec: 104.972
INFO:tensorflow:loss = 0.018230295, step = 50200 (0.953 sec)
INFO:tensorflow:global_step/sec: 104.661
INFO:tensorflow:loss = 0.018264093, step = 50300 (0.955 sec)
INFO:tensorflow:global_step/sec: 106.036
INFO:tensorflow:loss = 0.02319906, step = 50400 (0.943 sec)
INFO:tensorflow:global_step/sec: 105.034
INFO:tensorflow:loss = 0.014284887, step = 50500 (0.952 sec)
INFO:tensorflow:global_step/sec: 105.843
INFO:tensorflow:loss = 0.020154292, step = 50600 (0.945 sec)
INFO:tensorflow:global_step/sec: 105.44
INFO:tensorflow:loss = 0.01658813, step = 50700 (0.949 sec)
INFO:tensorflow:global_step/sec: 106.031
INFO:tensorflow:loss = 0.017267017, step = 50800 (0.943 sec)
INFO:tensorflow:global_step/sec: 105.795
INFO:tensorflow:loss = 0.026146078, step = 50900 (0.945 sec)
INFO:tensorflow:global_step/sec: 105.46
INFO:tensorflow:loss = 0.016701704, step = 51000 (0.948 sec)
INFO:tensorflow:global_step/sec: 106.237
INFO:tensorflow:loss = 0.01531472, step = 511

INFO:tensorflow:global_step/sec: 106.528
INFO:tensorflow:loss = 0.01872412, step = 58300 (0.939 sec)
INFO:tensorflow:global_step/sec: 105.038
INFO:tensorflow:loss = 0.012223836, step = 58400 (0.952 sec)
INFO:tensorflow:global_step/sec: 106.025
INFO:tensorflow:loss = 0.019247677, step = 58500 (0.943 sec)
INFO:tensorflow:global_step/sec: 105.079
INFO:tensorflow:loss = 0.022807121, step = 58600 (0.951 sec)
INFO:tensorflow:global_step/sec: 105.362
INFO:tensorflow:loss = 0.015921883, step = 58700 (0.949 sec)
INFO:tensorflow:global_step/sec: 106.216
INFO:tensorflow:loss = 0.02006164, step = 58800 (0.942 sec)
INFO:tensorflow:global_step/sec: 106.789
INFO:tensorflow:loss = 0.012790796, step = 58900 (0.936 sec)
INFO:tensorflow:global_step/sec: 106.193
INFO:tensorflow:loss = 0.013842871, step = 59000 (0.942 sec)
INFO:tensorflow:global_step/sec: 105.784
INFO:tensorflow:loss = 0.019313062, step = 59100 (0.946 sec)
INFO:tensorflow:global_step/sec: 106.509
INFO:tensorflow:loss = 0.011087801, step = 

INFO:tensorflow:global_step/sec: 105.39
INFO:tensorflow:loss = 0.018365644, step = 64900 (0.949 sec)
INFO:tensorflow:global_step/sec: 105.509
INFO:tensorflow:loss = 0.013316197, step = 65000 (0.948 sec)
INFO:tensorflow:global_step/sec: 105.558
INFO:tensorflow:loss = 0.0081987325, step = 65100 (0.947 sec)
INFO:tensorflow:global_step/sec: 105.184
INFO:tensorflow:loss = 0.0137282945, step = 65200 (0.951 sec)
INFO:tensorflow:global_step/sec: 105.788
INFO:tensorflow:loss = 0.016196365, step = 65300 (0.945 sec)
INFO:tensorflow:global_step/sec: 105.435
INFO:tensorflow:loss = 0.016590627, step = 65400 (0.949 sec)
INFO:tensorflow:global_step/sec: 106.913
INFO:tensorflow:loss = 0.010919217, step = 65500 (0.935 sec)
INFO:tensorflow:global_step/sec: 105.368
INFO:tensorflow:loss = 0.010630905, step = 65600 (0.949 sec)
INFO:tensorflow:global_step/sec: 104.876
INFO:tensorflow:loss = 0.019063687, step = 65700 (0.954 sec)
INFO:tensorflow:global_step/sec: 106.449
INFO:tensorflow:loss = 0.013466141, step

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [8]:
# average the results
logps_martel = np.array(logps_martel)
logps_martel_ave = np.mean(logps_martel,axis=0)
#logps_martel_ave

logps_star = np.array(logps_star)
logps_star_ave = np.mean(logps_star,axis=0)

logps_nonstar = np.array(logps_nonstar)
logps_nonstar_ave = np.mean(logps_nonstar, axis=0)

In [9]:
# get the true value of martel
logp_exp_martel  = []
for x in results_martel:
    logp_exp_martel.append(x[3])

# get the true value of star
logp_exp_star = []
for x in results_star:
    logp_exp_star.append(x[3])
    
# get the true value of nonstar
logp_exp_nonstar = []
for x in results_nonstar:
    logp_exp_nonstar.append(x[3])

In [10]:
# calculate the mae, mse, and rmse of martel
mae_martel = mean_absolute_error(logps_martel_ave,logp_exp_martel)
mse_martel = mean_squared_error(logps_martel_ave,logp_exp_martel)
mae_martel,mse_martel,np.sqrt(mse_martel)

(0.8689620071289579, 1.3487006364093497, 1.1613357121906438)

In [11]:
# calculate the mae, mse, and rmse of star
mae_star = mean_absolute_error(logps_star_ave,logp_exp_star)
mse_star = mean_squared_error(logps_star_ave,logp_exp_star)
mae_star,mse_star,np.sqrt(mse_star)

(0.5854634188593197, 0.7040420916522107, 0.8390721611710227)

In [12]:
# calculate the mae, mse, and rmse of nonstar
mae_nonstar = mean_absolute_error(logps_nonstar_ave,logp_exp_nonstar)
mse_nonstar = mean_squared_error(logps_nonstar_ave,logp_exp_nonstar)
mae_nonstar,mse_nonstar,np.sqrt(mse_nonstar)

(0.1982158786628176, 0.12593034492324373, 0.3548666579480858)